In [ ]:
!pip install transformers torch pandas nltk tqdm pdfplumber

In [ ]:
import os
import re
import nltk
import pandas as pd
import pdfplumber

nltk.data.path = [
    os.path.expanduser(r"C:\Users\vikas\AppData\Roaming\nltk_data")
]

# Ensure tokenizer resource exists
nltk.download("punkt", quiet=True)

# Explicit tokenizer import (avoids cached loader bug)
from nltk.tokenize.punkt import PunktSentenceTokenizer


In [3]:
text = ""

with pdfplumber.open("constitution.pdf") as pdf:
    for page in pdf.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text + "\n"

len(text)


853190

In [4]:
# Basic whitespace cleanup
text = re.sub(r"\n+", "\n", text)
text = re.sub(r"\s+", " ", text)

print(type(text))
print(len(text))


<class 'str'>
853190


In [5]:
tokenizer = PunktSentenceTokenizer()
sentences = tokenizer.tokenize(text)

len(sentences)


8319

In [6]:
def is_valid_sentence(s):
    s = s.strip()
    if len(s) < 40:
        return False
    # Remove lines with too many non-ASCII characters (garbled Hindi headers)
    if sum(c.isascii() for c in s) / len(s) < 0.85:
        return False
    # Remove dotted separator lines
    if set(s) == {"."}:
        return False
    return True

clean_sentences = [s for s in sentences if is_valid_sentence(s)]

len(clean_sentences)


3766

In [7]:
# Ensure output directory exists
os.makedirs("../data", exist_ok=True)

df = pd.DataFrame(clean_sentences, columns=["sentence"])
df.to_csv("../data/constitution_chunks.csv", index=False)

df.head(10)


,sentence
0,"£ÉÉ®iÉ BÉEÉ ºÉÆÉÊ´ÉvÉÉxÉ [1 , 2024 ] THE CONST..."
1,"In this edition, the text of the Constitution ..."
2,The foot notes below the text indicate the Con...
3,The Constitution (One Hundredth Amendment) Act...
4,The Constitution (Application to Jammu and Kas...
5,"New Delhi; Dr.Rajiv Mani, 1stMay, 2024 Secreta..."
6,.................................................
7,.................................................
8,.................................................
9,.................................................


In [8]:
print("Total raw sentences:", len(sentences))
print("Total clean sentences:", len(clean_sentences))
print("\nSample sentences:\n")

for s in clean_sentences[:5]:
    print("-", s)


Total raw sentences: 8319
Total clean sentences: 3766

Sample sentences:

- £ÉÉ®iÉ BÉEÉ ºÉÆÉÊ´ÉvÉÉxÉ [1 , 2024 ] THE CONSTITUTION OF INDIA [As on 1st May, 2024] 2024 GOVERNMENT OF INDIA MINISTRY OF LAW AND JUSTICE LEGISLATIVE DEPARTMENT,OFFICIAL LANGUAGES WING PREFACE This is the sixth pocket size edition of the Constitution of India in the diglot form.
- In this edition, the text of the Constitution of India has been brought up-to-date by incorporating therein all the amendments up to the Constitution (One Hundred and Sixth Amendment) Act, 2023.
- The foot notes below the text indicate the Constitution Amendment Acts by which such amendmentshave been made.
- The Constitution (One Hundredth Amendment) Act, 2015 containing details of acquired and transferred territories between the Governments of India and Bangladesh has been provided in AppendixI.
- The Constitution (Application to Jammu and Kashmir) Order, 2019 and the declaration under article 370(3) of the Constitution have been pro